# Digital Twin Data Acquisition

In this notebook, we will use Big Query and The Graph to obtain the signals required for the Rai Digital Twin. We will get state snapshots and calculate the absolute changes between time periods. 


### Original Signals  (Depreciated)
* Price signals:
    * Eth price
    * Rai price in eth - need new source
    * Rai price in usd - need new source
    * Redemption price
    * Redemption rate.
* Uniswap data - https://thegraph.com/explorer/subgraph/uniswap/uniswap-v2
* Total debt = sum over SAFEs of debt
* Total collateral = sum over SAFEs of Collateral
* Total rai = directly observable from the RAI erc 20 contract (not sure how to get it from theGraph), further note that total rai = principle debt
* Excess Liability = Total debt - total rai
* ETH in Uniswap = should be queriable from thegraph data on uniswap (different source subgraph) --> future replace this with total value in ETH of assets RAI is paired with on Uniswap instances
* RAI in Uniswap = should be queriable from thegraph data on uniswap
* Debt ceiling = directly observable from RAI subgraph
* debt as fraction of debt ceiling = total debt/debt ceiling
* floating RAI = total rai - rai in uniswap
* floating RAI as fraction of total RAI = floating RAI /total RAI
another thing i am interested in from uniswap is the yield per unit liquidity
* total liquidity = price_usd_asset1 * quantity_of_asset1 + price_usd_asset2*quantity_of_asset2
* volume_per_period = take directly from contract
* fees_per_period = measure directly or compute from volumes
* revenues in token1 and token2
* value of profits in USD
* yields = profit in usd per total liquidity in USD of the whole pool
* Ratio of yields to total liquidity


## Current: Mapping of specification states to data
The quantity state variables of the system are:
* ETH in collateral = collateral
* ETH in Uniswap = reserve1
* RAI in Uniswap = reserve0 
* RAI drawn from SAFEs = erc20CoinTotalSupply

The metric state variables of the system are:

* Market Price of RAI in ETH = marketPriceEth
* Market Price of RAI in USD = marketPriceUsd
* Market Price of ETH in USD = ETH Price (OSM)

The metric control variables of the system are:

* Redemption Price of RAI in USD = RedemptionPrice
* Redemption Price of RAI in ETH = RedemptionPriceinEth Calculate based off of ETH Price (OSM)?

In [1]:
# import libraries
import os
from google.cloud import bigquery
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import json
import requests
# The Graph URL
url = 'https://api.thegraph.com/subgraphs/name/reflexer-labs/rai-mainnet'

%matplotlib inline

# constants
constant = 1000000000000000000

#defining creditionals
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/aclarkdata/Downloads/raidata-5e1723d42cf5.json'

# initializing agent
client = bigquery.Client()


In [2]:
# SQL query
sql = """
SELECT * 
FROM `blockchain-etl.ethereum_rai.ChainlinkMedianETHUSD_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_next_OSM = client.query(sql).to_dataframe()
eth_price_next_OSM['ETH Price (Next OSM)'] = eth_price_next_OSM['medianPrice'].astype(float)/constant
# subset
eth_price_next_OSM = eth_price_next_OSM[['block_number','ETH Price (Next OSM)']]

eth_price_next_OSM.head()

,block_number,ETH Price (Next OSM)
0,12271233,2129.458042
1,12270946,2219.590000
2,12270590,2244.796301
3,12270259,2236.288682
4,12269966,2218.569789


In [3]:
# SQL query
sql = """
SELECT 
* 
FROM `blockchain-etl.ethereum_rai.OSM_event_UpdateResult`
ORDER By block_timestamp DESC

"""

eth_price_OSM = client.query(sql).to_dataframe()
eth_price_OSM['ETH Price (OSM)'] = eth_price_OSM['newMedian'].astype(float)/constant
# subset
eth_price_OSM = eth_price_OSM[['block_number','ETH Price (OSM)']]

eth_price_OSM.head()

,block_number,ETH Price (OSM)
0,12388416,3565.549850
1,12388143,3521.749280
2,12387897,3484.967617
3,12387633,3509.940000
4,12387345,3438.463957


In [4]:
# blocknumbers = eth_price_OSM.block_number.values.tolist()

# # look for any weth/rai pair - eventually multiple pairs in future versions
# uniswap_url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
# pair = []
# for i in blocknumbers[-5:-1]:
#     query = '''
#     {
#       pairs(block: {number:%s}, where: {id: "0x8ae720a71622e824f576b4a8c03031066548a3b1"}){
#         id,
#         token0 {
#           id
#         },
#         token1 {
#           id
#         }
#         reserve0,
#         reserve1,
#         totalSupply,
#         reserveETH,
#         reserveUSD,
#         token0Price,
#         token1Price
#       }
#     }
    
#     ''' % i
#     r = requests.post(uniswap_url, json = {'query':query})
#     s = json.loads(r.content)['data']['pairs'][0]
#     pair.append(s)
        
# pairState = pd.DataFrame(pair)

# #pairState['block_number'] = blocknumbers


# pairState.head()



In [5]:
blocknumbers = eth_price_OSM.block_number.values.tolist()

state = []
for i in blocknumbers[:-5]:
    query = '''
    {
      systemState(block: {number:%s},id:"current") { 
        coinUniswapPair {
          label
          reserve0
          reserve1
          token0Price
          token1Price
          totalSupply
        }
        currentCoinMedianizerUpdate{
          value
        }
        currentRedemptionRate {
          eightHourlyRate
          annualizedRate
          hourlyRate
          createdAt
        }
        currentRedemptionPrice {
          value
        }
        erc20CoinTotalSupply
        globalDebt
        globalDebtCeiling
        safeCount,
        totalActiveSafeCount
        coinAddress
        wethAddress
        systemSurplus
        debtAvailableToSettle
        lastPeriodicUpdate
        createdAt
        createdAtBlock
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['systemState']
    state.append(s)
        
systemState = pd.DataFrame(state)

systemState['block_number'] = blocknumbers[:-5]


systemState.head()


,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
0,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '13...",1529678381,5834717,{'value': '3.030114432731030231'},{'value': '3.005443242548134717184603198'},{'annualizedRate': '0.978703628689180357673951...,0,26115712.201303224796912264,26424432.5414415655529856168380246,115792089237316195423570985008687.9,1620406863,1966,230055.3942801043943153310278136914,583,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12388416
1,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '13...",1529678381,5834717,{'value': '3.030114432731030231'},{'value': '3.005443242548134717184603198'},{'annualizedRate': '0.978703628689180357673951...,0,26015712.201303224796912264,26324641.40437663388995386983992175,115792089237316195423570985008687.9,1620400447,1966,230002.3649514594642822328174048461,583,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12388143
2,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '13...",1529678381,5834717,{'value': '3.030114432731030231'},{'value': '3.005449713016319751797714048'},{'annualizedRate': '0.978703628689180357673951...,0,26015712.201303224796912264,26324641.40437663388995386983992175,115792089237316195423570985008687.9,1620400447,1966,230002.3649514594642822328174048461,583,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12387897
3,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '13...",1529678381,5834717,{'value': '3.030114432731030231'},{'value': '3.005464888103675014633099262'},{'annualizedRate': '0.978703628689180357673951...,0,25985712.201303224796912264,26294624.65886648852605071098430801,115792089237316195423570985008687.9,1620396470,1966,229927.7168168293955261746060588251,583,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12387633
4,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '13...",1529678381,5834717,{'value': '3.019126215606478828'},{'value': '3.005464888103675014633099262'},{'annualizedRate': '0.978703628689180357673951...,0,25985712.201303224796912264,26294624.65886648852605071098430801,115792089237316195423570985008687.9,1620391405,1966,229927.7168168293955261746060588251,583,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,12387345


In [6]:
systemState.tail(15)

,coinAddress,coinUniswapPair,createdAt,createdAtBlock,currentCoinMedianizerUpdate,currentRedemptionPrice,currentRedemptionRate,debtAvailableToSettle,erc20CoinTotalSupply,globalDebt,globalDebtCeiling,lastPeriodicUpdate,safeCount,systemSurplus,totalActiveSafeCount,wethAddress,block_number
1933,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '10...",1529678381,5834717,{'value': '17.513757357283865436'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,6256.799518562104188503,7204.985987855580927079986421601194,115792089237316195423570985008687.9,1613442165,5,101.727636135314939354760839476389,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11865563
1934,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '12...",1529678381,5834717,{'value': '17.513757357283865436'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,6256.799518562104188503,7204.985987855580927079986421601194,115792089237316195423570985008687.9,1613442165,5,101.727636135314939354760839476389,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11865271
1935,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '12...",1529678381,5834717,{'value': '17.513757357283865436'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,6256.799518562104188503,7204.841129160410399042582689488635,115792089237316195423570985008687.9,1613439664,5,101.7060524026427969159799621820111,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11865022
1936,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '12...",1529678381,5834717,{'value': '25.91762953962810453'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,6256.799518562104188503,7204.841129160410399042582689488635,115792089237316195423570985008687.9,1613431354,5,101.7060524026427969159799621820111,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11864748
1937,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '12...",1529678381,5834717,{'value': '25.91762953962810453'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,6256.799518562104188503,7204.841129160410399042582689488635,115792089237316195423570985008687.9,1613431354,5,101.7060524026427969159799621820111,6,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11864535
1938,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '77...",1529678381,5834717,{'value': '25.91762953962810453'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,5456.799518562104188503,6404.824426640505566612968254590487,115792089237316195423570985008687.9,1613424459,4,101.6899208386121073771205705459673,5,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11864197
1939,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '81...",1529678381,5834717,{'value': '25.91762953962810453'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,5456.799518562104188503,6404.824426640505566612968254590487,115792089237316195423570985008687.9,1613424459,4,101.6899208386121073771205705459673,5,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11863946
1940,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWAP_POOL_COIN', 'reserve0': '81...",1529678381,5834717,{'value': '0'},{'value': '3.14'},"{'annualizedRate': '1', 'createdAt': '16132260...",846.1745167717419600463715557405962,5456.799518562104188503,6404.824426640505566612968254590487,115792089237316195423570985008687.9,1613420437,4,101.6899208386121073771205705459673,5,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,11863692
1941,0x03ab458634910aad20ef5f1c8ee96f1d6ac54919,"{'label': 'UNISWA

In [7]:
## reserve 0= rai
## reserve 1 = eth

# The quantity state variables of the system are:
# * ETH in collateral = collateral
# * ETH in Uniswap = reserve1
# * RAI in Uniswap = reserve0 
# * RAI drawn from SAFEs = erc20CoinTotalSupply
systemState.coinUniswapPair.values[-100:-99][0]

{'label': 'UNISWAP_POOL_COIN',
 'reserve0': '9006136.159600056258735356',
 'reserve1': '16271.530235618410158973',
 'token0Price': '553.4904234074805907771404419173915',
 'token1Price': '0.001806715993103639120714508071342709',
 'totalSupply': '372625.851091413498251694'}

In [8]:
systemState['RedemptionRateAnnualizedRate'] = systemState.currentRedemptionRate.apply(lambda x: x['annualizedRate'])    
systemState['RedemptionRateHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['hourlyRate'])
systemState['RedemptionRateEightHourlyRate'] = systemState.currentRedemptionRate.apply(lambda x: x['eightHourlyRate'])
systemState['RedemptionPrice'] = systemState.currentRedemptionPrice.apply(lambda x: x['value'])
systemState['EthInUniswap'] = systemState.coinUniswapPair.apply(lambda x: x['reserve1'])
systemState['RaiInUniswap'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
systemState['RaiDrawnFromSAFEs'] = systemState['erc20CoinTotalSupply']
#systemState['RAIInUniswapV2(RAI/ETH)'] = systemState.coinUniswapPair.apply(lambda x: x['reserve0'])
del systemState['currentRedemptionRate']
del systemState['currentRedemptionPrice']
#systemState['RAIInUniswapV2(RAI/ETH)'] = systemState['RAIInUniswapV2(RAI/ETH)'].astype(float)


# subset
# systemState = systemState[['debtAvailableToSettle','erc20CoinTotalSupply','globalDebt',
#                            'globalDebtCeiling','systemSurplus','totalActiveSafeCount',
#                            'block_number', 'RedemptionRateAnnualizedRate',
#                            'RedemptionRateHourlyRate', 'RedemptionRateEightHourlyRate',
#                         'RedemptionPrice', 'RAIInUniswapV2(RAI/ETH)']]
systemState = systemState[['debtAvailableToSettle','globalDebt',
                           'globalDebtCeiling','systemSurplus','totalActiveSafeCount',
                           'block_number', 'RedemptionRateAnnualizedRate',
                           'RedemptionRateHourlyRate', 'RedemptionRateEightHourlyRate',
                        'RedemptionPrice', 'EthInUniswap','RaiInUniswap','RaiDrawnFromSAFEs']]

In [9]:
hourly = []
for i in blocknumbers[1:-5]:
    query = '''
    {
      hourlyStats(where: {blockNumber_gt: %s}) { 
        marketPriceUsd # price of COIN in USD (uni pool price * ETH median price)
        marketPriceEth # Price of COIN in ETH (uni pool price)
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['hourlyStats'][0]
    hourly.append(s)
        
hourlyStats = pd.DataFrame(hourly)

hourlyStats['block_number'] = blocknumbers[1:-5]

hourlyStats.head()


,marketPriceEth,marketPriceUsd,block_number
0,0.0008581767827068208010584367586379256,3.051839692894169064516002216700605,12388143
1,0.0008581767827068208010584367586379256,3.051839692894169064516002216700605,12387897
2,0.0008691030157179216530398150097739949,3.060762919763508963117586156700778,12387633
3,0.0008703163183947308644424523596888884,3.051667789193842215350231730280333,12387345
4,0.0008752624963036233420352468294409797,3.043224160894557444801878919780065,12387095


In [10]:
safehistories = []
for i in blocknumbers[:-5]:
    query = '''
    {
      safes(block: {number:%s}) {
            collateral
            debt
      }
    }
    ''' % i
    r = requests.post(url, json = {'query':query})
    s = json.loads(r.content)['data']['safes']
    t = pd.DataFrame(s)
    t['collateral'] = t['collateral'].astype(float)
    t['debt'] = t['debt'].astype(float)
    safehistories.append(pd.DataFrame(t.sum().to_dict(),index=[0]))

safe_history = pd.concat(safehistories)


In [11]:
safe_history.head()

,collateral,debt
0,1249.4226,293469.366076
0,1249.4226,293469.366076
0,1249.4226,293469.366076
0,1249.4226,293469.366076
0,1249.4226,293469.366076


In [12]:
safe_history['block_number'] = blocknumbers[:-5]
safe_history.reset_index(inplace=True)
del safe_history['index']

In [13]:
states = hourlyStats.merge(systemState,how='inner',on='block_number')
states = states.merge(safe_history,how='inner',on='block_number')

In [14]:
states = states.merge(eth_price_OSM,how='inner',on='block_number')

In [15]:
states.head()

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,EthInUniswap,RaiInUniswap,RaiDrawnFromSAFEs,collateral,debt,ETH Price (OSM)
0,0.0008581767827068208010584367586379256,3.051839692894169064516002216700605,12388143,0,26324641.40437663388995386983992175,115792089237316195423570985008687.9,230002.3649514594642822328174048461,583,0.978703628689180357673951782,0.999997542650177411470860719,0.99998034137049836900859785,3.005443242548134717184603198,11977.04886394660783202,13892364.888112882038435692,26015712.201303224796912264,1249.4226,293469.366076,3521.749280
1,0.0008581767827068208010584367586379256,3.051839692894169064516002216700605,12387897,0,26324641.40437663388995386983992175,115792089237316195423570985008687.9,230002.3649514594642822328174048461,583,0.978703628689180357673951782,0.999997542650177411470860719,0.99998034137049836900859785,3.005449713016319751797714048,12025.302339733495657016,13836452.206761711690220331,26015712.201303224796912264,1249.4226,293469.366076,3484.967617
2,0.0008691030157179216530398150097739949,3.060762919763508963117586156700778,12387633,0,26294624.65886648852605071098430801,115792089237316195423570985008687.9,229927.7168168293955261746060588251,583,0.978703628689180357673951782,0.999997542650177411470860719,0.99998034137049836900859785,3.005464888103675014633099262,12033.68070829093980817,13826789.701572709287985804,25985712.201303224796912264,1249.4226,293469.366076,3509.940000
3,0.0008703163183947308644424523596888884,3.051667789193842215350231730280333,12387345,0,26294624.65886648852605071098430801,115792089237316195423570985008687.9,229927.7168168293955261746060588251,583,0.978703628689180357673951782,0.999997542650177411470860719,0.99998034137049836900859785,3.005464888103675014633099262,12033.68070829093980817,13826789.701572709287985804,25985712.201303224796912264,1249.4226,293469.366076,3438.463957
4,0.0008752624963036233420352468294409797,3.043224160894557444801878919780065,12387095,0,26294073.9625992015325165098259239,115792089237316195423570985008687.9,229903.0410604860638363149273366366,583,0.978703628689180357673951782,0.999997542650177411470860719,0.99998034137049836900859785,3.005471631471800371100367975,12080.165571477830468286,13774235.96457323700068437,25985212.201303224796912264,1249.4226,293469.366076,3464.125574


In [16]:
states.tail(20)

,marketPriceEth,marketPriceUsd,block_number,debtAvailableToSettle,globalDebt,globalDebtCeiling,systemSurplus,totalActiveSafeCount,RedemptionRateAnnualizedRate,RedemptionRateHourlyRate,RedemptionRateEightHourlyRate,RedemptionPrice,EthInUniswap,RaiInUniswap,RaiDrawnFromSAFEs,collateral,debt,ETH Price (OSM)
1927,0.003961862840112759595111040785451548,7.138834472759815214522785312231516,11866918,846.1745167717419600463715557405962,8005.063097217343907641456094814183,115792089237316195423570985008687.9,101.7661908161964296350666365039205,8,1,1,1,3.14,3.189046246129420508,804.93605529227666153,6254.514977346455183755,33.99336,7157.848499,1797.054268
1928,0.003961862840112759595111040785451548,7.138834472759815214522785312231516,11866661,846.1745167717419600463715557405962,8005.063097217343907641456094814183,115792089237316195423570985008687.9,101.7661908161964296350666365039205,8,1,1,1,3.14,3.151419087240594731,813.968197826446049584,6254.514977346455183755,33.99336,7157.848499,1805.760000
1929,0.003871673482644513537411971393784648,6.95760735640489718366929627361697,11866377,846.1745167717419600463715557405962,8004.909406251387781073606551202994,115792089237316195423570985008687.9,101.7451032740474329040709042772893,7,1,1,1,3.14,3.149093447887812591,813.968197826446049584,6254.514977346455183755,33.99236,7157.848499,1805.760000
1930,0.003871673482644513537411971393784648,6.95760735640489718366929627361697,11866094,846.1745167717419600463715557405962,8004.909406251387781073606551202994,115792089237316195423570985008687.9,101.7451032740474329040709042772893,7,1,1,1,3.14,2.630169209633951588,974.047036860082824364,7056.799518562104188503,33.99236,7157.848499,1816.150000
1931,0.002700248663670810750113398394548099,4.876001026910203220124770284939175,11865824,846.1745167717419600463715557405962,8004.909406251387781073606551202994,115792089237316195423570985008687.9,101.7451032740474329040709042772893,7,1,1,1,3.14,2.430169209633951588,1053.964708368352970279,7056.799518562104188503,33.99236,7157.848499,1816.150000
1932,0.002305740591064103441107518006356187,4.193534028137562623793662011493303,11865563,846.1745167717419600463715557405962,7204.985987855580927079986421601194,115792089237316195423570985008687.9,101.727636135314939354760839476389,6,1,1,1,3.14,2.430169209633951588,1053.964708368352970279,6256.799518562104188503,31.89236,6357.960015,1798.907804
1933,0.002305740591064103441107518006356187,4.193534028137562623793662011493303,11865271,846.1745167717419600463715557405962,7204.985987855580927079986421601194,115792089237316195423570985008687.9,101.727636135314939354760839476389,6,1,1,1,3.14,2.130169209633951588,1201.933726709008306816,6256.799518562104188503,31.89236,6357.960015,1796.160000
1934,0.001772285078867473883528342992003263,3.180649139640300679812976034028576,11865022,846.1745167717419600463715557405962,7204.841129160410399042582689488635,115792089237316195423570985008687.9,101.7060524026427969159799621820111,6,1,1,1,3.14,2.130169209633951588,1201.933726709008306816,6256.799518562104188503,31.89236,6357.960015,1765.955398
1935,0.001772285078867473883528342992003263,3.180649139640300679812976034028576,11864748,846.1745167717419600463715557405962,7204.841129160410399042582689488635,115792089237316195423570985008687.9,101.7060524026427969159799621820111,6,1,1,1,3.14,2.130169209633951588,1201.933726709008306816,6256.799518562104188503,31.89236,6357.960015,1795.404693
1936,0.001772285078867473883528342992003263,3.180649139640300679812976034028576,11864535,846.1745167717419600463715557405962,7204.841129160410399042582689488635,115792089237316195423570985008687.9,101.7060524026427969159799621820111,6,1,1,1,3.14,2.130169209633951588,1201.933726709008306816,6256.799518562104188503,31.89236,6357.960015,1787.587273


In [17]:
states = states.astype(float)

In [18]:
# export
states.to_csv('states.csv')